In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import os
import string


KeyboardInterrupt



In [ ]:
log_dir = "../evaluation_results/runtime"
logs = pd.concat([pd.read_csv(os.path.join(log_dir, f), delimiter="; ") for f in os.listdir(log_dir) if os.path.isfile(os.path.join(log_dir, f))], axis=0)

In [ ]:
logs["t_matching [s]"].replace({-1: np.nan}, inplace=True)

In [ ]:
nn_runtime_logs = logs.drop(["t_matching [s]"], axis=1)

In [ ]:
n_var_values = nn_runtime_logs["n_var"].unique()

In [ ]:
nn_runtime_logs

In [ ]:
gt_runtime_logs = logs.dropna().drop(["n_var", "t_NN [s]"], axis=1)
gt_runtime_logs.sort_values(["k", "n_obs"])

In [ ]:
palette = sns.color_palette("Spectral", len(logs["k"].unique()))

In [ ]:
len(n_var_values)

In [ ]:
f, axs = plt.subplots(1, len(n_var_values) + 1, figsize=(12, 4), sharex=True, width_ratios=[1, 1, 1, 1, 1,1.5], sharey=True)
legend_idx = 3
y_lim_max = nn_runtime_logs["t_NN [s]"].max() + 100

for i, n_var in enumerate(n_var_values):
    legend = False if i!=legend_idx else "full"
    sns.barplot(nn_runtime_logs[nn_runtime_logs["n_var"] == n_var], x="n_obs", y="t_NN [s]", hue="k", ax=axs[i], palette=palette, legend=legend)
    if i == legend_idx:
        axs[i].legend(loc='upper center', title="k", bbox_to_anchor=(0.5, 1.3), ncol=len(gt_runtime_logs["k"].unique()))
    if i == 0:
        axs[i].set_ylabel("Runtime NN matching [s]")
    else:
        axs[i].set_ylabel(None)
        axs[i].set_yticklabels([])
    axs[i].set_xlabel("$n_{{obs}}$")
    axs[i].set_title(f"$n_{{vars}}={n_var}$")

sns.barplot(gt_runtime_logs, x="n_obs", y="t_matching [s]", hue="k", ax=axs[-1], palette=palette, legend=False)
#axs[-1].legend(loc='upper left', title="k")
axs[-1].set_ylabel("Runtime MWMCM [s]")
axs[-1].set_xlabel("$n_{{obs}}$")
axs[-1].set_yscale("symlog", linthresh=100) 

plt.subplots_adjust(wspace=0.05)

pos = axs[-1].get_position()  # Get current position
axs[-1].set_position([pos.x0 + 0.1, pos.y0, pos.width, pos.height])  # Shift right by 0.05

labels = string.ascii_lowercase
for i in range(len(n_var_values) + 1):
    axs[i].text(
        -0.05, 1.1,  # Position (normalized figure coordinates)
        labels[i+6],   # Corresponding letter
        transform=axs[i].transAxes,  # Relative to subplot
        fontsize=10, fontweight='bold', va='top', ha='left'
    )
    axs[i].tick_params(axis='x', labelrotation=90)
#plt.savefig("../plots/fig2/fig2-2.pdf", bbox_inches="tight")